# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [43]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [ ]:
# command text used to import 'establishments.json'

#mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json

In [44]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [45]:
# Select the database and collection
db = mongo['uk_food']
collection = db['establishments_collection']

In [46]:
# Query and print documents from the collection
for document in collection.find():
    pprint(document)

In [47]:
# confirm that our new database was created
print(mongo.list_database_names())



['admin', 'autosaurus', 'classDB', 'config', 'gardenDB', 'local', 'uk_food']


In [48]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [49]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [51]:
# review a document in the establishments collection
print(db['establishments'].find_one())

{'_id': ObjectId('66f8ac3fb1720f347db359f1'), 'FHRSID': 254410, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000053944', 'BusinessName': 'St Margarets At Cliffe C P School', 'BusinessType': 'School/college/university', 'BusinessTypeID': 7845, 'AddressLine1': 'Sea Street', 'AddressLine2': 'St Margarets At Cliffe', 'AddressLine3': 'Kent', 'AddressLine4': '', 'PostCode': 'CT15 6SS', 'Phone': '', 'RatingValue': '5', 'RatingKey': 'fhrs_5_en-gb', 'RatingDate': '2019-01-16T00:00:00', 'LocalAuthorityCode': '182', 'LocalAuthorityName': 'Dover', 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/', 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk', 'scores': {'Hygiene': 0, 'Structural': 5, 'ConfidenceInManagement': 0}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '1.375095', 'latitude': '51.152913'}, 'RightToReply': '', 'Distance': 4587.840617011452, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCount': 0, 'returncode':

In [52]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [53]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "New Cafe",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "AddressLine1": "123 High Street",
    "LocalAuthorityName": "Sample Authority",
    "RatingValue": "4",
    "geocode": {
        "longitude": "1.2345",
        "latitude": "5.6789"
    }
}

In [54]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)

InsertOneResult(ObjectId('66f8adeb650c76c36693e655'), acknowledged=True)

In [55]:
# Check that the new restaurant was inserted
pprint(establishments.find_one({"BusinessName": "New Cafe"}))

{'AddressLine1': '123 High Street',
 'BusinessName': 'New Cafe',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'LocalAuthorityName': 'Sample Authority',
 'RatingValue': '4',
 '_id': ObjectId('66f8adeb650c76c36693e655'),
 'geocode': {'latitude': '5.6789', 'longitude': '1.2345'}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [56]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
business_type = establishments.find_one({"BusinessType": "Restaurant/Cafe/Canteen"}, {"BusinessTypeID": 1, "BusinessType": 1})
pprint(business_type)

{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('66f8ac3fb1720f347db359f3')}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [57]:
# Update the new restaurant with the correct BusinessTypeID
if business_type:
    establishments.update_one({"BusinessName": "New Cafe"}, {"$set": {"BusinessTypeID": business_type['BusinessTypeID']}})

In [58]:
# Confirm that the new restaurant was updated
pprint(establishments.find_one({"BusinessName": "New Cafe"}))

{'AddressLine1': '123 High Street',
 'BusinessName': 'New Cafe',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'LocalAuthorityName': 'Sample Authority',
 'RatingValue': '4',
 '_id': ObjectId('66f8adeb650c76c36693e655'),
 'geocode': {'latitude': '5.6789', 'longitude': '1.2345'}}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [59]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"Number of documents with LocalAuthorityName 'Dover': {dover_count}")

Number of documents with LocalAuthorityName 'Dover': 994


In [60]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many({"LocalAuthorityName": "Dover"})

DeleteResult({'n': 994, 'ok': 1.0}, acknowledged=True)

In [61]:
# Check if any remaining documents include Dover
dover_remaining = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"Remaining documents with LocalAuthorityName 'Dover': {dover_remaining}")

Remaining documents with LocalAuthorityName 'Dover': 0


In [62]:
# Check that other documents remain with 'find_one'
pprint(establishments.find_one())

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('66f8ac3fb1720f347db35cd7'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': 

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [63]:
# Change the data type from String to Decimal for longitude and latitude
from bson.decimal128 import Decimal128

establishments.update_many({}, [{
    '$set': {
        'geocode.longitude': {'$toDecimal': "$geocode.longitude"},
        'geocode.latitude': {'$toDecimal': "$geocode.latitude"}
    }
}])

UpdateResult({'n': 38786, 'nModified': 38786, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

Use `update_many` to convert `RatingValue` to integer numbers.

In [67]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [66]:
# Change the data type from String to Integer for RatingValue
establishments.update_many(
    {"RatingValue": {"$exists": True, "$ne": None}},
    [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}]
)

WriteError: Failed to parse number 'AwaitingInspection' in $convert with no onError value: Did not consume whole string., full error: {'index': 0, 'code': 241, 'errmsg': "Failed to parse number 'AwaitingInspection' in $convert with no onError value: Did not consume whole string."}

In [65]:
# Check that the coordinates and rating value are now numbers
pprint(establishments.find_one({"BusinessName": "New Cafe"}))

{'AddressLine1': '123 High Street',
 'BusinessName': 'New Cafe',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'LocalAuthorityName': 'Sample Authority',
 'RatingValue': '4',
 '_id': ObjectId('66f8adeb650c76c36693e655'),
 'geocode': {'latitude': Decimal128('5.6789'),
             'longitude': Decimal128('1.2345')}}
